<a href="https://colab.research.google.com/github/benjaminbrown038/torch/blob/main/torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

from torch.nn import functional
from torch.nn import Sequential, Conv2d, MaxPool2d, Dropout, Flatten, Linear, Softmax, CrossEntropyLoss
from torch.optim import SGD, Adam
from torch import optim
from torch.utils.data import DataLoader
from torch.optim import SGD


import torchvision

from torchvision import models
from torchvision import transforms
from torchvision.models import resnet50, alexnet, alexnet, inception_v3
from torch.nn import Sequential

from torchsummary import summary

import matplotlib.pyplot as plt
import numpy
import os
import cv2

In [29]:
%%capture

mnist_train = torchvision.datasets.MNIST(root = '/data', download = True,train = True,transform = transforms.ToTensor())
mnist_test = torchvision.datasets.MNIST(root = '/data', download = True, train = False, transform = transforms.ToTensor())

mnist_train_dataloader = DataLoader(mnist_train,batch_size = 64,shuffle = True)
mnist_test_dataloader = DataLoader(mnist_test,batch_size = 64, shuffle = True)


mnist_train1 = torchvision.datasets.MNIST(root = '/data', download = True,train = True,transform = transforms.Compose([transforms.ToTensor(),transforms.Grayscale(3)]))
mnist_test1 = torchvision.datasets.MNIST(root = '/data', download = True, train = False, transform = transforms.Compose([transforms.ToTensor(),transforms.Grayscale(3)]))

mnist_train_dataloader1 = DataLoader(mnist_train,batch_size = 64,shuffle = True)
mnist_test_dataloader1 = DataLoader(mnist_test,batch_size = 64, shuffle = True)


cifar_train = torchvision.datasets.MNIST(root = '/data', download = True, train = True)
cifar_test = torchvision.datasets.MNIST(root = '/data',download = True, train = False)


cifar_train_dataloader = DataLoader(mnist_train, batch_size = 64, shuffle = True)
cifar_test_dataloader = DataLoader(mnist_test, batch_size = 64, shuffle = True)


In [3]:
model = Sequential(
            Conv2d(1, 32, kernel_size=3),
            Conv2d(32, 64, kernel_size=3),
            MaxPool2d(2),
            Dropout(0.25),
            Flatten(),
            Linear(9216, 128),
            Linear(128, 10),
            Softmax())
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
         MaxPool2d-3           [-1, 64, 12, 12]               0
           Dropout-4           [-1, 64, 12, 12]               0
           Flatten-5                 [-1, 9216]               0
            Linear-6                  [-1, 128]       1,179,776
            Linear-7                   [-1, 10]           1,290
           Softmax-8                   [-1, 10]               0
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.66
Params size (MB): 4.58
Estimated Total Size (MB): 5.24
----------------------------------------------------------------


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [7]:
model1 = Sequential(
		Conv2d(1,32,kernel_size = 2),
		MaxPool2d(2,2),
		Conv2d(32,64,kernel_size = 2),
		MaxPool2d(2,2),
		Conv2d(64,128,kernel_size = 2),
		Dropout(0.25),
		MaxPool2d(1,1),
		Conv2d(128,10,kernel_size = 3),
		Flatten(),
		Linear(90,10))
summary(model1,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 27, 27]             160
         MaxPool2d-2           [-1, 32, 13, 13]               0
            Conv2d-3           [-1, 64, 12, 12]           8,256
         MaxPool2d-4             [-1, 64, 6, 6]               0
            Conv2d-5            [-1, 128, 5, 5]          32,896
           Dropout-6            [-1, 128, 5, 5]               0
         MaxPool2d-7            [-1, 128, 5, 5]               0
            Conv2d-8             [-1, 10, 3, 3]          11,530
           Flatten-9                   [-1, 90]               0
           Linear-10                   [-1, 10]             910
Total params: 53,752
Trainable params: 53,752
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.38
Params size (MB): 0.21
Estimated Tot

In [8]:
model2 = Sequential(
		Conv2d(1,32,kernel_size = 3),
		Conv2d(32,64,kernel_size = 3),
		MaxPool2d(2),
		Dropout(0.25),
	  Flatten(),
		Linear(9216,128),
		Linear(128,10))
summary(model2,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
         MaxPool2d-3           [-1, 64, 12, 12]               0
           Dropout-4           [-1, 64, 12, 12]               0
           Flatten-5                 [-1, 9216]               0
            Linear-6                  [-1, 128]       1,179,776
            Linear-7                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.66
Params size (MB): 4.58
Estimated Total Size (MB): 5.24
----------------------------------------------------------------


In [34]:
%%capture
pre_trained_one = resnet50(pretrained = True)
summary(pre_trained_one,(3,28,28))

In [37]:
%%capture
pre_trained_two = inception_v3(pretrained=True)
summary(pre_trained_two,(3,28,28))

RuntimeError: ignored

In [38]:
%%capture
pre_trained_three = alexnet(pretrained = True)
summary(pre_trained_three,(3,28,28))

RuntimeError: ignored

In [18]:
def train_loop(dataloader,model,loss_fn,optimizer):
	size = len(dataloader.dataset)
	for batch, (x,y) in enumerate(dataloader):
		pred = model(x)
		loss = loss_fn(pred,y)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		if batch % 100 == 0:
			loss,current = loss.item(), batch*len(x)
			print("Loss: ",loss,current)

In [19]:
loss = CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr = 0.001,momentum = 0.9)
optimizer1 = optim.SGD(model1.parameters(),lr=0.001,momentum = 0.9)
optimizer2 = optim.SGD(model2.parameters(), lr =0.001,momentum = 0.9)
optimizer3 = optim.SGD(pre_trained_one.parameters(),lr=0.001,momentum = 0.9)
optimizer4 = optim.SGD(pre_trained_two.parameters(),lr=0.001,momentum=0.9)
optimizer5 = optim.SGD(pre_trained_three.parameters(),lr=0.001,momentum=0.9)

In [20]:
train_loop(mnist_train_dataloader,model,loss,optimizer)

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


2.2965948581695557 0
2.2784268856048584 6400
2.2674801349639893 12800
2.210325002670288 19200
2.0878727436065674 25600
1.9821844100952148 32000
1.813801884651184 38400
1.7161420583724976 44800
1.7413032054901123 51200
1.6127793788909912 57600


In [21]:
train_loop(mnist_train_dataloader,model1,loss,optimizer)

2.2995951175689697 0
2.306455612182617 6400
2.3051178455352783 12800
2.307050943374634 19200
2.2922420501708984 25600
2.3168370723724365 32000
2.309736728668213 38400
2.312749147415161 44800
2.294609546661377 51200
2.3153648376464844 57600


In [22]:
train_loop(mnist_train_dataloader,model2,loss,optimizer)

2.295835018157959 0
2.3027875423431396 6400
2.3123998641967773 12800
2.3014447689056396 19200
2.2993216514587402 25600
2.304833173751831 32000
2.3112845420837402 38400
2.2946877479553223 44800
2.3102166652679443 51200
2.2986104488372803 57600


In [23]:
train_loop(mnist_train_dataloader,model,loss,optimizer)
train_loop(mnist_train_dataloader,model2,loss,optimizer)

1.6093798875808716 0
1.748428225517273 6400
1.6382205486297607 12800
1.606386423110962 19200
1.6898581981658936 25600
1.594357967376709 32000
1.6385828256607056 38400
1.5650193691253662 44800
1.578779935836792 51200
1.6348260641098022 57600
2.304631471633911 0
2.295976400375366 6400
2.29062819480896 12800
2.319627523422241 19200
2.2844810485839844 25600
2.3122103214263916 32000
2.314455986022949 38400
2.315218448638916 44800
2.2985496520996094 51200
2.2926156520843506 57600


In [30]:
train_loop(mnist_train_dataloader1,pre_trained_one,loss,optimizer)

RuntimeError: ignored

In [ ]:
train_loop(mnist_train_dataloader,pre_trained_two,loss,optimizer)

IndexError: ignored

In [ ]:
train_loop(mnist_train_dataloader,pre_trained_three,loss,optimizer)

RuntimeError: ignored